In [1]:
#imports
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas
from PIL import Image
import os
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import glob
import cv2
import time as tm
import keras_tuner as kt
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [9]:
# ==========================================
# ===== ADD YOUR SAVED MODEL PATH HERE =====
# ==========================================

SAVED_MODEL_DIR = '[YOUR PATH]'

In [11]:
# ============================================
# ===== SET PUZZLE PIECE IMAGE DIMS HERE =====
# ============================================
DIM = 20

x_shape = (DIM,DIM,6)

In [26]:
def build_ANN(hp):
    #create model object
    model = Sequential()
    #adding first convolutional layer    
    model.add(Conv2D(
        #adding filter 
        filters=hp.Int('conv_0_filter', min_value=32, max_value=128, step=16),
        # adding filter size or kernel size
        kernel_size=(3,3),
        #activation function
        activation=hp.Choice("activation", ["relu", "tanh"]),
        input_shape=x_shape))
        
    model.add(Dropout(rate=hp.Float(
                'dropout_conv_0',
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )))
    
    
    for i in range(hp.Int("num_layers", 1, 3)):
        #Adding nth convlutional layer
        model.add(Conv2D(
            #adding filter 
            filters=hp.Int('conv_{}_filter'.format(i+1), min_value=32, max_value=128, step=16),
            #adding filter size or kernel size
            kernel_size=(3,3),
            #activation function
            activation=hp.Choice("activation", ["relu", "tanh"])))
        
        model.add(Dropout(rate=hp.Float(
                'dropout_conv_{}'.format(i+1),
                min_value=0.0,
                max_value=0.5,
                default=0.25,
                step=0.05,
            )))
    
    
    
    #adding flatten layer 
    model.add(Flatten())
    
    #adding fully connected layer
    model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), activation=hp.Choice("activation", ["relu", "tanh"])))
    model.add(Dropout(rate=hp.Float(
                    'dropout_0',
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05,
                )))
              
    
    for i in range(hp.Int("num_fully_connected_layers", 0, 3)):
        model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16), activation=hp.Choice("activation", ["relu", "tanh"])))
        #adds final dropout layer
        model.add(Dropout(rate=hp.Float(
                    'dropout_{}'.format(i+1),
                    min_value=0.0,
                    max_value=0.5,
                    default=0.25,
                    step=0.05,
                )))
    
    #output
    model.add(Dense(5, activation='softmax'))

    
    #chooses optimizer
    optimizer = hp.Choice('optimizer', ['adam', 'sgd'])
    
    #compilation of model
    model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 
    #return model 
    return model

In [27]:
# X = np.random.random((10,*x_shape))
# y = np.random.random((10,5))

In [28]:
X = pass
y = pass

SyntaxError: invalid syntax (3787615945.py, line 1)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [30]:
#buids model 
model = build_ANN(kt.HyperParameters())
#creates tuner
tuner = kt.BayesianOptimization(
    hypermodel=build_ANN,
    objective="val_accuracy",
    max_trials=200,
    executions_per_trial=10,
    overwrite=True,
)
#gets best models 
tuner.search(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

## folder path to save models in --- CHANGE AS NEEDED 
folder = SAVED_MODEL_DIR

#gets the top 10 hyperparemeters
best_hyps = tuner.get_best_hyperparameters(10)

#gets the top 10 models, trains them, and saves them in a folder 
for i in range(len(best_hyps)):
    #get best ith hyper params 
    best_hp = best_hyps[i]
    #build model from hyperparamters 
    model=tuner.hypermodel.build(best_hp)
    #trains model
    model.fit(X_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)
    #gets model accuracy 
    _, accuracy = model.evaluate(X_test, y_test)
        
    #saves model in file 
    file_name = str(i) + str(accuracy)
    model.save(folder + file_name)

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.5
Total elapsed time: 00h 01m 18s


KeyboardInterrupt: 

In [10]:
# #buids model 
# model = build_ANN(kt.HyperParameters())
# #creates tuner
# tuner = kt.RandomSearch(
#     hypermodel=build_ANN,
#     objective="val_accuracy",
#     max_trials=10,
#     executions_per_trial=5,
#     overwrite=True,
# )
# #gets best models 
# tuner.search(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

# ## folder path to save models in --- CHANGE AS NEEDED 
# folder = '/Users/maxperozek/ML-CP341/jigsaw/MachineLearningTeamProject/Model/Saved_models/rand-hyp-search'

# #gets the top 10 hyperparemeters
# best_hyps = tuner.get_best_hyperparameters(10)

# #gets the top 10 models, trains them, and saves them in a folder 
# for i in range(len(best_hyps)):
#     #get best ith hyper params 
#     best_hp = best_hyps[i]
#     #build model from hyperparamters 
#     model=tuner.hypermodel.build(best_hp)
#     #trains model
#     model.fit(X_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)
#     #gets model accuracy 
#     _, accuracy = model.evaluate(X_test, y_test)
        
#     #saves model in file 
#     file_name = str(i) + str(accuracy)
#     model.save(folder + file_name)

## Model saving

In [14]:
save_path = '/Users/maxperozek/Desktop/90.0'

In [15]:
model.save(save_path)

INFO:tensorflow:Assets written to: /Users/maxperozek/Desktop/90.0/assets


In [16]:
loaded_model = keras.models.load_model(save_path)

In [17]:
loaded_model.predict(X)

array([[0.28220114, 0.21492095, 0.17501388, 0.17335993, 0.15450403],
       [0.2242131 , 0.21804485, 0.20514414, 0.1836821 , 0.16891587],
       [0.19518158, 0.22487716, 0.20327194, 0.20086163, 0.17580777],
       [0.22675285, 0.2058932 , 0.17495728, 0.20920902, 0.18318765],
       [0.22291896, 0.22926109, 0.20009089, 0.18206282, 0.16566625],
       [0.23319536, 0.22888558, 0.17729418, 0.18374106, 0.17688388],
       [0.24491885, 0.21808548, 0.15842941, 0.20276652, 0.1757997 ],
       [0.1876505 , 0.24106799, 0.2062972 , 0.20523243, 0.15975183],
       [0.22426337, 0.2351623 , 0.19669743, 0.19300167, 0.15087521],
       [0.256212  , 0.22196357, 0.1732413 , 0.1789814 , 0.16960175]],
      dtype=float32)